In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import pandas as pd
import librosa
#import librosa.display
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from datetime import datetime
import asyncio
import functools
import concurrent.futures
from tqdm.notebook import tqdm

#settings
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.config.set_logical_device_configuration(physical_devices[0],[tf.config.LogicalDeviceConfiguration(memory_limit=3900)])
# tf.config.set_visible_devices(physical_devices[1:], 'GPU')
import cmsisdsp as dsp
import cmsisdsp.mfcc as mfcc
from cmsisdsp.datatype import F32
import scipy.signal as sig

In [ ]:
df = pd.read_csv("/home/birdo/MachineLearning/Data/bird_sounds.csv")

In [ ]:
# mffc init
mfccf32=dsp.arm_mfcc_instance_f32()
sample_rate = 16000
sample_time = 10
sample_count = sample_rate * sample_time
FFTSize = 512
Hopsamp = int(FFTSize / 2)
numOfDctOutputs = 32
freq_min = 64
freq_high = sample_rate / 2
numOfMelFilters = 48
row_size = int((sample_count - FFTSize) / Hopsamp)
print(row_size)

window = sig.hamming(FFTSize, sym=False)
filtLen,filtPos,packedFilters = mfcc.melFilterMatrix(F32,freq_min, freq_high, numOfMelFilters,sample_rate,FFTSize)
dctMatrixFilters = mfcc.dctMatrix(F32,numOfDctOutputs, numOfMelFilters)
status=dsp.arm_mfcc_init_f32(mfccf32,FFTSize,numOfMelFilters,numOfDctOutputs,dctMatrixFilters,filtPos,filtLen,packedFilters,window)
print(status)



In [ ]:
species_ids = df.species_id.unique()
species_dict = dict(zip(species_ids, range(len(species_ids))))
tempcopy = df.copy()
df["species_index"] = df["species_id"]
tempcopy = tempcopy.applymap(lambda s: species_dict.get(s) if s in species_dict else s)#.convert_objects("int64")
df["species_index"] = tempcopy["species_id"]

In [ ]:
df["file_path"] = df["species_id"]+ "/" + df["recording_id"]   
basePath = "/home/birdo/MachineLearning/Data/"
sound_filePath = "/home/birdo/MachineLearning/clean-audio/"

In [ ]:
df

## Feature generating functions

In [ ]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desirex width
    :return: padded array
    """
    h = array.shape[0]
    w = array.shape[1]
    a = max((xx - h) // 2,0)
    aa = max(0,xx - a - h)
    b = max(0,(yy - w) // 2)
    bb = max(yy - b - w,0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [ ]:


def generate_features(y_cut, sr):
    tmp=np.zeros(FFTSize + 2)
    sample_count = len(y_cut)
    row_count = (sample_count - FFTSize) / Hopsamp
    col_count = numOfDctOutputs
    MFCCs = np.zeros((row_size, col_count))
    

    for i in range(0, int(row_count)):
        start_index = i * Hopsamp
        input = y_cut[start_index:start_index + FFTSize]
        output = dsp.arm_mfcc_f32(mfccf32,input,tmp)
        MFCCs[i, :] = output[0:col_count]

    # MFCC_padded = padding(MFCCs, 310, 32)
    # print(MFCC_padded.shape)
    # print(MFCCs.shape)
    # print(MFCCs)
    # print(MFCCs.shape)

    return MFCCs

In [ ]:
def writeLabel(_labelPath,label):
    f = open(_labelPath, "w")
    f.write(str(label))
    f.close()

def saveImage(_imgPath,data):
    data.tofile(_imgPath)

async def gen_feature_from_name_id(_trainfolder,_filename,_specie_name,_recording_id,_i,executor=None):
    # write label file
    labelPath = basePath + str("/Labels/") + _trainfolder + _specie_name + str("-") + _recording_id
    # check if file exists
    imgPath = basePath + str("/Images-512/") + _trainfolder + _specie_name + str("-") + _recording_id


    # replace .wav with .npy
    imgPath = imgPath.replace(".wav", ".npy")
    labelPath = labelPath.replace(".wav", ".npy")


    # check if files dont contain the word .nfs
    if ".nfs" in imgPath or ".nfs" in labelPath:
        return
    
    loop = asyncio.get_event_loop()

    # check if label exists
    if not os.path.isfile(labelPath):
        #write label file
        await loop.run_in_executor(executor,functools.partial(writeLabel,labelPath,_i))

    # check if image exists
    if os.path.isfile(imgPath):
        return

    #Load the file
    y, sr = await loop.run_in_executor(executor,functools.partial(librosa.load,_filename,sr=None))
    #cut the file to signal start and end  
    y_cut=y 
    #generate features & output numpy array          
    data = await loop.run_in_executor(executor,functools.partial(generate_features,y_cut, sr))
    # print type of numpy array
    # print(data.dtype)
    # print shape of numpy array
    # print(data.shape)
    
    #save the numpy array
    await loop.run_in_executor(executor,functools.partial(saveImage,imgPath, data))

async def get_features(df_in, trainfolder,executor=None):   
    features=[]     
    labels = [] #empty array to store labels  
    output=[]
    #For each species, determine how many augmentations are needed
    df_in=df_in.reset_index()     
    for i in df_in.species_index.unique():
        print('species:',i)    
        #all the file indices with the same species_id     
        filelist = df_in.loc[df_in.species_index == i].index
        filelist_coroutines = []
        for j in range(0,len(filelist)):   
            try:
                filename = str(sound_filePath) + df_in.iloc[filelist[j]].file_path #.file_path.values[j] #get the file name 
                #define the beginning time of the signal
                specie_name = df_in.iloc[filelist[j]].species_id
                recording_id = df_in.iloc[filelist[j]].recording_id

                filelist_coroutines.append(gen_feature_from_name_id(trainfolder,filename,specie_name,recording_id,i,executor=executor))
            except FileNotFoundError as e:
                print(e)
            except Exception as e:
                print(e)
            else:
                continue
            
        complete_count = 0
        print(df_in.species_id.unique())

        for f in tqdm(asyncio.as_completed(filelist_coroutines), total=len(filelist_coroutines),mininterval=0.01,maxinterval=10, miniters=1):
            result = await f
            complete_count += 1

In [ ]:
X=df.drop("species_index", axis=1)
y=df.species_index

In [ ]:
#Split once to get the test and training set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123, stratify=y)
#Split twice to get the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=123)
print(X_train.shape, X_test.shape, X_val.shape, len(y_train), len(y_test), len(y_val))

In [ ]:
#use get_features to calculate and store the features
with concurrent.futures.ThreadPoolExecutor(max_workers=8,thread_name_prefix="birds") as pool:
    batch_size = 2500
    # batch count
    batch_count = len(X) // batch_size
    print(f"batch count: {batch_count}")
    for i in range(0, len(X), batch_size):
        await get_features(pd.concat([X[i:i+batch_size], y[i:i+batch_size]], axis=1), "all/",pool)
        print(f"features {((i/batch_size)/batch_count)*100}% done")
        # await asyncio.sleep(1)

    # print("mini_test done")
    # await get_features(pd.concat([X_train[:300], y_train[:300]], axis=1), "mini_train/",pool)
    # print("mini_train done")
    # await get_features(pd.concat([X_test,y_test],axis=1), "test/",pool)
    # print("test done")
    # await get_features(pd.concat([X_val,y_val],axis=1), "val/",pool)
    # print("val done")
    # await get_features(pd.concat([X_train,y_train],axis=1), "train/",pool)
    # print("train done")